# Chapter 1
## The Python Data Model
The Python interpreter invokes special methods to perform basic object operations. These special methods are written with leading and trailing double underscores. An example is the syntax when accessing a dictionary value with obj[key], this is supported by the __getitem__ special method. In order to evaluate my_dict[key], the interpreter calls my_dict.__getitem__(key). These methods are also referred to as magic methods and dunder methods (double underscore before and after).

## A Pythonic Deck of Cards

In [2]:
# Example 1.1
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self) -> None:
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self) -> int:
        return len(self._cards)
    
    def __getitem__(self, position) -> list:
        return self._cards[position]

We use collections.namedtuple to constuct a class to represent individual cards. namedtuple is use to build class objects that are just attributes with no custom methods

In [3]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [4]:
deck = FrenchDeck()
len(deck)

52

In [5]:
print(deck[0])
print(deck[-1])

Card(rank='2', suit='spades')
Card(rank='A', suit='hearts')


In [6]:
from random import choice
for i in range(4):
    print(choice(deck))

Card(rank='9', suit='spades')
Card(rank='A', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='10', suit='spades')


Because our __getitem__ delegates to the list operator of self._cards our deck automatically supports slicing:

In [8]:
print(deck[:3])
print(deck[12::13])
for card in deck:
    print(card)

[Card(rank='2', suit='spades'), Card(rank='3', suit='spades'), Card(rank='4', suit='spades')]
[Card(rank='A', suit='spades'), Card(rank='A', suit='diamonds'), Card(rank='A', suit='clubs'), Card(rank='A', suit='hearts')]
Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')
Card(rank='7', suit='spades')
Card(rank='8', suit='spades')
Card(rank='9', suit='spades')
Card(rank='10', suit='spades')
Card(rank='J', suit='spades')
Card(rank='Q', suit='spades')
Card(rank='K', suit='spades')
Card(rank='A', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='3', suit='diamonds')
Card(rank='4', suit='diamonds')
Card(rank='5', suit='diamonds')
Card(rank='6', suit='diamonds')
Card(rank='7', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='K', suit='diamonds')
Card

In [12]:
# Our deck class is also iterable and therefore the 'in' operator will perform a sequential scan over the object
print(Card('Q', 'hearts') in deck)
print(Card('F', 'hearts') in deck)


True
False


## How Special Methods are Used
Special methods like __len__ and __getitem__ are called by the Python interpreter. However the interpreter takes a shortcut when dealing with built-in types like list, str or extensions like the NumPy arrays. Python objects include a structure called PyVarObject, whish has an ob_size field holding the number of items in the collection., so if your_object is an instance of one of the built-in methods, then len(your_object) retrieves the value of the ob_size filed, which is a faster way of calling the method.

## Emulating(Copying) Numeric Types
Illustrating the use of special methods by implementing a class to represent two-dimensional vectors (Euclidean Vectors).


In [8]:
import math

class Vector:
    
    def __init__(self, x=0, y=0) -> None:
        self.x = x
        self.y = y

    def __repr__(self) -> str:
        return f'Vector({self.x!r}, {self.y!r})'
    
    def __abs__(self) -> float:
        return math.hypot(self.x, self.y)
    
    def __bool__(self) -> bool:
        return bool(abs(self))
    
    def __add__(self, other) -> type[any]:
        x = self.x + other.x
        y =self.y + other.y
        return Vector(x, y)
    
    def __mul__(self, scalar) -> type[any]:
        return Vector(self.x * scalar, self.y * scalar)


In [12]:
v1 = Vector(2, 4)
v2 = Vector(2, 1)
print(v1 + v2)
print(v1.__str__()) # works because if no __str__ defined will default to __repr__

Vector(4, 5)
Vector(2, 4)


## String Representations
The __repr__ dunder method is called by repr built-in to get the string representation of the object for inspection. Without a custom __repr__ the console would display a Vector instance <Vector objet at 0x10e100070>.

Note the f-string in the __repr__ uses `!r` to get the standard representation of the attributes displayed. It displays the object as the initial type the variable was set at `Vector(1, 2)` instead of `Vector('1', '2')`.

__str__ is called by the str() built in and implicitly used by the print funtion. Note that there is one default which is true: if __repr__ is defined, and __str__ is not, the object will behave as though __str__=__repr__. View [What is the difference between __str__ and __repr__ in Python](https://fpy.li/1-5) for an explanation of the difference.

## Boolean Value of a Custom Type
By default, instances of user-defined classes are considered truthy (truthy and falsy are used to differentiate from bool values True and False). bool(x) calls x.__bool__() and uses the result. If __bool__ is not implemented Python tries x.__len__() and if that returns zero bool returns False. Else it returns True.